# Generate Infinite Tiktoks

## Generate video text
The first step to generate a tiktok, is obviously to know what you're going to stay.

To do so, my idea was to find something in genius.com, the famous website to have extra information on rap/pop songs. You can have informations about the context of the song when it was released.


In [2]:
import os
import requests


In [3]:
def get_annotation(annotation_id):
    url = f"https://api.genius.com/annotations/{annotation_id}"
    payload = {}
    headers = {'Authorization': 'Bearer ' + os.environ['RAPGENIUS_API_KEY']}
    r = requests.request("GET", url, headers=headers, data=payload)
    data = r.json()
    
    full_annotation = ""
    parts = data['response']['annotation']['body']['dom']['children']

    full_annotation = find_annotation(parts)
    
    return full_annotation
def find_annotation(root):
    # print(type(root))
    full_annotation = ""
    if(type(root) == str):
        if(root == ""):
            root = '\n'
        full_annotation += root
    elif(type(root) == dict):
        if('children' in root):
            full_annotation += find_annotation(root['children'])
    elif(type(root) == list):
        for part in root:
            full_annotation += find_annotation(part)
    
    return full_annotation
def get_id_song(url):
    response = requests.get(url).content
    id = int(response.split(b'genius://songs/')[1].split(b'"')[0])
    return id
def get_song_description(song_id):
    url = f"https://api.genius.com/songs/{song_id}"

    payload = {}
    headers = {
        'Authorization': 'Bearer ' + os.environ['RAPGENIUS_API_KEY']
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    data = response.json()
    return get_annotation(data['response']['song']['description_annotation']['id'])


In [4]:
url = "https://genius.com/Eminem-houdini-lyrics"

song_description = get_song_description(get_id_song(url))
print(song_description)

“Houdini” is the lead single of Eminem’s twelfth studio album The Death of Slim Shady (Coup de Grâce). The track is named after Harry Houdini, a popular magician known for his death-defying stunts like the Chinese Water Torture Cell, which Eminem replicates in the intro to his 1999 track, “Role Model.”
With the return of long-time collaborator Jeff Bass of the Bass Brothers, Eminem revisits his catchy albeit morbid sound of the early 2000s. This fanfare track sees him callback to his old catchphrases such as Shady’s iconic “Guess who’s back?” line from the 2002 track “Without Me.” The song heavily samples and interpolates the Steve Miller Band’s 1982 track “Abracadabra.”
On October 24, 1926, the legendary Houdini tragically passed away in Detroit, Michigan at the age of 52. Interestingly, Detroit is also the hometown of Eminem, who turns 52 in October 2024.
The single was first teased by Eminem on May 20, 2024, when he posted a teaser to his social media accounts with a text message fr

## Create Audio for the video

In [10]:
from openai import OpenAI
import os

api_key = os.environ.get('OPENAI_API_KEY_LORRAIN')
client = OpenAI(api_key=api_key)

def text_to_speech_mp3(text, output_filename, model="tts-1", voice="alloy"):
    """
    Converts a text string to speech using the OpenAI API and writes the resulting audio to an MP3 file.

    Args:
    text (str): The text string to convert to speech.
    output_filename (str): The name of the output MP3 file.
    model (str): The model to use for speech synthesis. Defaults to 'tts-1'.
    voice (str): The voice to use for speech synthesis. Defaults to 'alloy'.
    """
    global client
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY_LORRAIN')) if client is None else client
    speech_file_path = output_filename


    with client.audio.speech.with_streaming_response.create(
        model=model,
        voice=voice,
        input=text,
    ) as response:
        response.stream_to_file(speech_file_path+('.mp3' if not speech_file_path.endswith('.mp3') else ''))

def getTextTimingsOfMp3(mp3file):
    global client
    client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY_LORRAIN')) if client is None else client
    
    audio_file = open(mp3file, "rb")
    transcript = client.audio.transcriptions.create(
        file=audio_file,
        model="whisper-1",
        response_format="verbose_json",
        timestamp_granularities=["word"]
    )

    return transcript


In [42]:
def onlychars(text):
    return ''.join(c if c.isalnum() or c.isspace() else '' for c in text)

def verify_transcription(original_text, transcription):
    """
    Verifies that the text is the same after the tts and stt.

    Args:
    original_text (str): The original text.
    transcription (dict): The transcription dictionary.

    Returns:
    int: ratio of the original text and the transcription text.
    """
    transcription_text = onlychars(transcription.text.lower())
    original_text = onlychars(original_text.lower())
    
    print('original :\n',original_text.replace('\n', ' '))
    print('transcription :\n',transcription_text,'\n')
    
    
    if transcription_text == original_text:
        return 1
    
    # correct_words = 0
    for word in original_text.split():
        # print(f'testing {word}')
        len_before = len(transcription_text)
        transcription_text = str(transcription_text).replace(word, '', 1)
        len_after = len(transcription_text)
        if(len_before == len_after):
            print(f'word {word} not found')
        # print(f'removed word ? {len_before - len_after}')
#
    print(len(original_text.split()), len(transcription_text.split()))
    return 1 - len(transcription_text.split()) / len(original_text.split())


In [11]:
text_to_speech_mp3(song_description, 'audio_description')
timing_description = getTextTimingsOfMp3('audio_description.mp3')

In [48]:
print(timing_description)
print(verify_transcription(song_description, timing_description) * 100, "%")

final_text = timing_description.text

Transcription(text="Houdini is the lead single of Eminem's 12th studio album, The Death of Slim Shady, Coup de Grace. The track is named after Harry Houdini, a popular magician known for his death-defying stunts like the Chinese water torture cell, which Eminem replicates in the intro to his 1999 track, Role Model. With the return of longtime collaborator Jeff Bass of the Bass Brothers, Eminem revisits his catchy albeit morbid sound of the early 2000s. This fanfare track sees him call back to his old catchphrases such as Shady's iconic Guess Who's Back line from the 2002 track Without Me. The song heavily samples and interpolates the Steve Miller band's 1982 track, Abracadabra. On October 24th, 1926, the legendary Houdini tragically passed away in Detroit, Michigan at the age of 52. Interestingly, Detroit is also the hometown of Eminem who turns 52 in October 2024. The single was first teased by Eminem on May 20th, 2024 when he posted a teaser to his social media accounts with a text m

## Find pictures

### Find picture descrition
The idea I came up with is to ask a LLM (mistral in my case) to find some nice moments in the description of the song

In [47]:
import os
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage

def get_description_pictures(story):
    api_key = os.environ["MISTRAL_API_KEY"]
    model = "open-mistral-7b"

    client = MistralClient(api_key=api_key)

    preprompt = """Your job will be to write the visual of a tiktok video: the images that show on screen. You have to find some nice moments in the story to place pictures that will show on screen. Tell me when you choose an image of what. In this format:
    DURING THE SENTENCE "[place the full sentence here]", SHOW AN IMAGE OF "[place the description of the image]".
    Answer only with the descriptions of all the images. 
    You can set as many images as you want (dont be shy ;)) at least 15, and if a sentence needs more than one image, you can split the sentence in two or more parts.
    Try not to give a description, but key words because i will look online for the exact same words you describe. That means if you are too complex, i will not be able to find the image.
    
    For example:
    DURING THE SENTENCE "he ate a sandwich", SHOW AN IMAGE OF "a sandwich".

    Now it is your turn:

    STORY : """

    messages = [
        ChatMessage(role="user", content=preprompt + f'"{story}"'),
    ]

    chat_response = client.chat(
        model=model,
        messages=messages,
    )

    results = chat_response.choices[0].message.content

    return results.split("\n\n")
def convert_to_dict(lines):
    visual_dict = {}
    for line in lines:
        print(line)
        if line:
            sentence, image_desc = line.split(', SHOW AN IMAGE OF ', 1)
            sentence = sentence.replace('DURING THE SENTENCE ', '', 1)
            visual_dict[sentence] = image_desc
    return visual_dict

def get_pictures(story):
    return convert_to_dict(get_description_pictures(story))


In [49]:
descriptions = get_pictures(final_text)
print(descriptions)

DURING THE SENTENCE "Houdini is the lead single of Eminem's 12th studio album, The Death of Slim Shady, Coup de Grace.", SHOW AN IMAGE OF "Eminem album cover - The Death of Slim Shady, Coup de Grace".
DURING THE SENTENCE "Coup de Grace being his final album.", SHOW AN IMAGE OF "Final album symbol".
DURING THE SENTENCE "The track is named after Harry Houdini, a popular magician known for his death-defying stunts like the Chinese water torture cell, which Eminem replicates in the intro to his 1999 track, Role Model.", SHOW AN IMAGE OF "Harry Houdini in Chinese water torture cell".
DURING THE SENTENCE "With the return of longtime collaborator Jeff Bass of the Bass Brothers, Eminem revisits his catchy albeit morbid sound of the early 2000s.", SHOW AN IMAGE OF "Eminem and Jeff Bass working together".
DURING THE SENTENCE "This fanfare track sees him call back to his old catchphrases such as Shady's iconic Guess Who's Back line from the 2002 track Without Me.", SHOW AN IMAGE OF "Eminem with t

### Generate or Find picture

In [50]:
from google_images_search import GoogleImagesSearch

def get_first_image_url(search_term, path_to_dir='images'):
    search_term = search_term.replace('"', '')
    # print(search_term)
    gis = GoogleImagesSearch(os.environ['GOOGLE_API_KEY'] , os.environ['GOOGLE_CX_KEY'])
    _search_params = {
        'q': search_term
    }
    gis.search(search_params=_search_params, path_to_dir=path_to_dir)

    for result in gis.results():
        return result.url

    return None

In [51]:
imgs_url = {}
for key in descriptions:
    # print(key)
    # print(descriptions[key])
    imgs_url[key] = get_first_image_url(descriptions[key], None)
    # break
print(imgs_url)

{'"Houdini is the lead single of Eminem\'s 12th studio album, The Death of Slim Shady, Coup de Grace."': 'https://images.genius.com/93a2e6a47a168080c985d3855f31b3ca.1000x563x1.png', '"Coup de Grace being his final album."': 'https://static.wikia.nocookie.net/wjsn/images/7/78/Queendom_2_FINAL_album_cover.jpg/revision/latest?cb=20220601095404', '"The track is named after Harry Houdini, a popular magician known for his death-defying stunts like the Chinese water torture cell, which Eminem replicates in the intro to his 1999 track, Role Model."': 'https://upload.wikimedia.org/wikipedia/commons/e/e8/Houdini_performing_Water_Torture_Cell.jpg', '"With the return of longtime collaborator Jeff Bass of the Bass Brothers, Eminem revisits his catchy albeit morbid sound of the early 2000s."': 'https://grantland.com/wp-content/uploads/2013/11/grant_g_eminemyoung_6402.jpg?w=750', '"This fanfare track sees him call back to his old catchphrases such as Shady\'s iconic Guess Who\'s Back line from the 20

In [ ]:
print("Images URL")
for key in imgs_url:
    print(key)
    print(imgs_url[key])

## Create video from the pictures

### Background Video

### Adding the audio

### Adding the pictures

In [ ]:
def get_sentence_timings(transcription, sentence):
    """
    Returns the start and end timings of a sentence in a transcription object.

    Args:
    transcription (Transcription): The transcription object.
    sentence (str): The sentence to search for.

    Returns:
    tuple: A tuple containing the start and end timings of the sentence.
    """
    words = transcription.words
    sentence_words = sentence.split()
    sentence_start = None
    sentence_end = None

    i = 0
    while i < len(words):
        if words[i]['word'].lower() == sentence_words[0].lower():
            j = 1
            while j < len(sentence_words) and i + j < len(words) and words[i + j]['word'].lower() == sentence_words[j].lower():
                j += 1
            if j == len(sentence_words):
                sentence_start = words[i]['start']
                sentence_end = words[i + j - 1]['end']
                break
        i += 1

    return sentence_start, sentence_end